In [8]:
# Étape 1: Installer les bibliothèques nécessaires
!pip install torch torchvision diffusers transformers huggingface_hub matplotlib scikit-image

  Using cached torchvision-0.20.0-cp311-cp311-win_amd64.whl.metadata (6.2 kB)
  Using cached scikit_image-0.24.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached torch-2.5.0-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached imageio-2.36.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached tifffile-2024.9.20-py3-none-any.whl.metadata (32 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
Using cached torchvision-0.20.0-cp311-cp311-win_amd64.whl (1.6 MB)
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   -----------------

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Sofyen Meddeb\Desktop\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Sofyen Meddeb\Desktop\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Sofyen Meddeb\Desktop\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Sofyen Meddeb\Desktop\venv\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Sofyen Meddeb\anaconda3\Lib\http\client.py", line 473, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\Sofyen Meddeb\anaconda3\Lib\socket.py", line 706,

In [10]:
!pip install torchvision

  Using cached torchvision-0.20.0-cp311-cp311-win_amd64.whl.metadata (6.2 kB)
  Using cached torch-2.5.0-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached torchvision-0.20.0-cp311-cp311-win_amd64.whl (1.6 MB)
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/203.1 MB 1.2 MB/s eta 0:02:50
   ---------------------------------------- 0.8/203.1 MB 1.0 MB/s eta 0:03:13
   ---------------------------------------- 1.0/203.1 MB 1.1 MB/s eta 0:03:05
   ---------------------------------------- 1.3/203.1 MB 1.1 MB/s eta 0:02:58
   ---------------------------------------- 1.6/203.1 MB 1.1 MB/s eta 0:02:56
   ---------------------------------------- 1.8/2

  You can safely remove it manually.


In [11]:
# Étape 2: Importer les bibliothèques
import torch
from PIL import Image
from torchvision import transforms
from skimage.color import rgb2lab, lab2rgb
import numpy as np
import matplotlib.pyplot as plt
from huggingface_hub import hf_hub_download

# Étape 3: Télécharger et charger le modèle GAN de colorisation
repo_id = "Hammad712/GAN-Colorization-Model"
model_filename = "generator.pt"
model_path = hf_hub_download(repo_id=repo_id, filename=model_filename)

# Fonction pour créer le générateur
def build_generator():
    from fastai.vision.learner import create_body
    from torchvision.models import resnet34
    from fastai.vision.models.unet import DynamicUnet
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    backbone = create_body(resnet34(), pretrained=True, n_in=1, cut=-2)
    G_net = DynamicUnet(backbone, 2, (256, 256)).to(device)
    return G_net

# Charger le modèle
G_net = build_generator()
G_net.load_state_dict(torch.load(model_path))

# Étape 4: Fonction de prétraitement de l'image
def preprocess_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img = transforms.Resize((256, 256))(img)
    img = np.array(img)
    img_to_lab = rgb2lab(img).astype("float32")
    img_to_lab = transforms.ToTensor()(img_to_lab)
    L = img_to_lab[[0], ...] / 50. - 1.
    return L.unsqueeze(0)

# Étape 5: Fonction de colorisation
def colorize_image(img_path, model):
    L = preprocess_image(img_path)
    with torch.no_grad():
        ab = model(L)
        L = (L + 1.) * 50.
        ab = ab * 110.
        Lab = torch.cat([L, ab], dim=1).permute(0, 2, 3, 1).cpu().numpy()
        return lab2rgb(Lab[0])

# Étape 6: Coloriser et afficher l'image
img_path = "path/to/your/image.jpg"  # Remplacer par le chemin de votre image en niveaux de gris
colorized_image = colorize_image(img_path, G_net)

# Afficher l'image colorisée
plt.imshow(colorized_image)
plt.axis("off")
plt.show()

# Enregistrer l'image colorisée
colorized_image = (colorized_image * 255).astype(np.uint8)
Image.fromarray(colorized_image).save("colorized_image.png")
print("L'image colorisée a été enregistrée sous 'colorized_image.png'")


RuntimeError: operator torchvision::nms does not exist

In [12]:
!pip install torch==2.0.0 torchvision==0.15.0


   ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/172.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/172.3 MB ? eta -:--:--
   ---------------------------------------- 0.5/172.3 MB 799.2 kB/s eta 0:03:35
   ---------------------------------------- 0.8/172.3 MB 907.1 kB/s eta 0:03:10
   ---------------------------------------- 1.0/172.3 MB 967.3 kB/s eta 0:02:58
   ---------------------------------------- 1.3/172.3 MB 1.0 MB/s eta 0:02:44
   ---------------------------------------- 1.6/172.3 MB 1.1 MB/s eta 0:02:37
   ---------------------------------------- 1.8/172.3 MB 1.1 MB/s eta 0:02:33
   ---------------------------------------- 2.1/172.3 MB 1.1 MB/s eta 0:02:30
    --------------------------------------- 2.4/172.3 MB 1.1 MB/s eta 0:02:30
    --------------------------------------- 2.6/172.3 MB 1.1 MB/s eta 0:02:29
    ----

Reason for being yanked: Contains an incorrect torch dependency
  You can safely remove it manually.
  You can safely remove it manually.


   ---------------------------- ---------- 127.9/172.3 MB 57.0 kB/s eta 0:12:59
   ---------------------------- ---------- 127.9/172.3 MB 57.0 kB/s eta 0:12:59
   ---------------------------- ---------- 127.9/172.3 MB 57.0 kB/s eta 0:12:59
   ---------------------------- ---------- 127.9/172.3 MB 57.0 kB/s eta 0:12:59
   ---------------------------- ---------- 127.9/172.3 MB 57.0 kB/s eta 0:12:59
   ---------------------------- ---------- 127.9/172.3 MB 57.0 kB/s eta 0:12:59
   ---------------------------- ---------- 127.9/172.3 MB 57.0 kB/s eta 0:12:59
   ---------------------------- ---------- 127.9/172.3 MB 57.0 kB/s eta 0:12:59
   ---------------------------- ---------- 127.9/172.3 MB 57.0 kB/s eta 0:12:59
   ---------------------------- ---------- 127.9/172.3 MB 57.0 kB/s eta 0:12:59
   ---------------------------- ---------- 127.9/172.3 MB 57.0 kB/s eta 0:12:59
   ---------------------------- ---------- 127.9/172.3 MB 57.0 kB/s eta 0:12:59
   ---------------------------- --------